# 1. Prueba para obtener mensajes desde GMAIL

In [57]:
# Importar la clase InstalledAppFlow de la biblioteca google_auth_oauthlib
from google_auth_oauthlib.flow import InstalledAppFlow
# Importar el módulo base64 para codificar y decodificar datos
import base64
# Para crear clientes de servicio para las APIs de Google
from googleapiclient.discovery import build  

# Función para manejar la autenticación de Gmail
def gmail_authenticate():
    # Definir el alcance (scope) para el acceso a Gmail, solo lectura en este caso
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

    # Configurar el flujo de autenticación con las credenciales de OAuth y el alcance definido
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    # Iniciar un servidor local para manejar la autenticación y obtener las credenciales
    creds = flow.run_local_server(port=0)

    # Devolver las credenciales obtenidas después de completar la autenticación
    return creds

# Función para recuperar los últimos 'n_mails' correos electrónicos
def get_last_n_emails(service, n_mails):
    # Solicitar la lista de mensajes para el usuario autenticado, limitado por 'n_mails'
    results = service.users().messages().list(userId='me', maxResults=n_mails).execute()
    # Obtener los mensajes de la respuesta de la API
    messages = results.get('messages', [])
    total_msg = []  # Lista para almacenar los mensajes procesados

    # Iterar sobre cada mensaje en la lista de mensajes
    for message in messages:
        # Obtener los detalles completos del mensaje usando su ID
        msg = service.users().messages().get(userId='me', id=message['id']).execute()

        # Extraer el payload y las partes del mensaje
        payload = msg.get('payload', {})
        parts = payload.get('parts', [])
        body = ""  # Variable para almacenar el cuerpo del mensaje
        # Verificar si hay datos codificados en base64 en el cuerpo del mensaje
        if 'data' in payload['body']:
            # Decodificar los datos del cuerpo del mensaje
            body = base64.urlsafe_b64decode(payload['body']['data']).decode("utf-8")
        else:
            # Iterar sobre las partes del mensaje para encontrar y decodificar el texto
            for part in parts:
                if part['mimeType'] == 'text/plain':
                    body = base64.urlsafe_b64decode(part['body']['data']).decode("utf-8")
                    break
        
        # Agregar el cuerpo del mensaje decodificado al diccionario del mensaje
        msg["body"] = body

        # Añadir el mensaje procesado a la lista total_msg
        total_msg.append(msg)
    # Devolver la lista de mensajes procesados
    return total_msg

# Función para obtener un valor específico de los encabezados de un correo electrónico
def get_value_for_key(email_headers, key):
    # Iterar sobre cada encabezado en los encabezados del correo electrónico
    for header in email_headers:
        # Verificar si el nombre del encabezado coincide con la clave buscada
        if header['name'] == key:
            # Devolver el valor del encabezado si se encuentra la clave
            return header['value']
    # Devolver None si no se encuentra la clave
    return None


def get_mails(n_mails):
    # Construir el cliente de servicio de Gmail con las credenciales autenticadas
    service = build('gmail', 'v1', credentials=gmail_authenticate())
    # Llamar a la función para obtener los últimos 10 correos electrónicos y almacenar la respuesta en 'msg'
    msg = get_last_n_emails(service, n_mails=n_mails)

    # Devolver los datos del mensaje obtenido
    return msg


## 1.1. Función para obtener mensajes

In [58]:
msg = get_mails(n_mails=3)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=694915007051-7ln9qfjl9jrakq6dtql35s1slenkhrl7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62417%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=NMlhNvG8LGfo5Mkc5YOPvqOl8ufw6t&access_type=offline


## 1.2. Respuesta individual de mensaje como diccionario

In [59]:
msg[0]

{'id': '18d4c17f35cc7cff',
 'threadId': '18d4c17f35cc7cff',
 'labelIds': ['UNREAD', 'CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'con su Seguro vida devolución Hola Jose, Esta es tu última oportunidad ¡para llevarte 1 vale de S/100! Compra hoy el Seguro Vida Devolución y además, podrás empezar a disfrutar de grandes beneficios',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'enriquemejiagamarra@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:7000:3528:b0:554:41f0:6ea8 with SMTP id t8csp281020mav;        Sat, 27 Jan 2024 10:04:14 -0800 (PST)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AGHT+IEGNyXhxe4mrOR3QchLHKtwIBoj7/jkcEtU10Tj3clPfdUyCyu4RbXYwbht1tMrK6IhPqWA'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:6358:7f13:b0:174:d596:d1ea with SMTP id p19-20020a0563587f1300b00174d596d1eamr1367685rwn.49.1706378654558;        Sat, 27 Jan 2024 10:04:14 -0800 (PST)'},
   {'name':

## 1.3. Obtener el asunto del correo

In [60]:
message_dict = dict(msg[0])
email_headers = message_dict['payload']['headers']
subject = get_value_for_key(email_headers, 'Subject')
subject

'👉 Vale de S/100 GRATIS para quienes quieran hacer crecer tus ahorros'

## 1.4. Obtener el contenido corto correo

In [61]:
content = message_dict['snippet']
content

'con su Seguro vida devolución Hola Jose, Esta es tu última oportunidad ¡para llevarte 1 vale de S/100! Compra hoy el Seguro Vida Devolución y además, podrás empezar a disfrutar de grandes beneficios'

## 1.5. Loop para hacer print a contenido y asunto de cada correo

In [41]:
for message in msg:
    message_dict = dict(message)

    email_headers = message_dict['payload']['headers']
    subject = get_value_for_key(email_headers, 'Subject')

    content = message_dict['snippet']
    print('###################')
    print(subject)
    print(content)

###################
Jose ¡Recuerda! El banco nunca te pedirá datos confidenciales
Hola Jose, En el BCP nos preocupamos por tu seguridad financiera y queremos que estés prevenido ante los posibles casos de fraude. Por eso, desde el BCP te brindamos estos consejos: No compartas tus
###################
Esta es una prueba
Hola, cómo estas? -- José Enrique Mejia Gamarra MACHINE LEARNING ENGINEER GERENCIA CORPORATIVA DE ESTRATEGIA Y DESARROLLO T. | A. | C. JR. CRISTOBAL DE PERALTA 820 , SANTIAGO DE SURCO - www.ferreycorp.
###################
IntelliSearch™ Alert found 12 new jobs, based on your profile 
Start applying today ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌
###################
Alguien ha añadido tu dirección de correo electrónico como su dirección de correo electrónico de recuperación
Alguien ha añadido enriquemejiagamarra@gmail.com como su direcci

## 2. Uso de Modelo pre-entrenado HuggingFace

In [1]:
# https://huggingface.co/openai-community/roberta-base-openai-detector
from transformers import pipeline
pipe = pipeline("text-classification", model="roberta-base-openai-detector")
print(pipe("Hello world! Is this content AI-generated?"))

C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


config.json: 100%|██████████| 624/624 [00:00<?, ?B/s] 
model.safetensors: 100%|██████████| 501M/501M [02:08<00:00, 3.91MB/s] 
Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 1.81MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.38MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<0

[{'label': 'Real', 'score': 0.8036580085754395}]
